In [1]:
#Needed Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import os
import sys

import networkx as nx

from tqdm import tqdm

import json
from pathlib import Path

import matplotlib.pyplot as plt

print("Starting Notebook.")

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

Starting Notebook.


In [2]:
tqdm.pandas()

In [3]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']
categories = ['General Relation', 'Duplication', 'Temporal/Causal', 'Composition', 'Workflow']

In [4]:
type_dict={'Backports': 'Workflow', 
           
    'Blocked': 'Temporal/Causal',
    'Blocker': 'Temporal/Causal',
    'Blocks': 'Temporal/Causal',
           
    'Bonfire Testing': 'Workflow', 
    'Bonfire testing': 'Workflow', 
    'Git Code Review': 'Workflow', 
    'Testing': 'Workflow',
           
    'Causality': 'Temporal/Causal', 
    'Cause': 'Temporal/Causal',
    'Caused': 'Temporal/Causal', 
    'Problem/Incident': 'Temporal/Causal',
           
    'Child-Issue': 'Composition', 
    'Parent Feature': 'Composition',
    'Parent/Child': 'Composition',
    'multi-level hierarchy [GANTT]': 'Composition',
    'Parent-Relation': 'Composition',
           
    'Cloners': 'Duplication', 
    'Cloners (old)': 'Duplication', 
           
    'Collection': 'Composition', 
    'Container': 'Composition', 
    'Contains(WBSGantt)': 'Composition', 
    'Incorporate': 'Composition', 
    'Incorporates': 'Composition', 
    'Part': 'Composition',
    'PartOf': 'Composition',
    'Superset': 'Composition', 
           
    'Completes': 'Workflow', 
    'Fixes': 'Workflow',
    'Resolve': 'Workflow',
           
    'Depend': 'Temporal/Causal', 
    'Dependency': 'Temporal/Causal', 
    'Dependent': 'Temporal/Causal', 
    'Depends': 'Temporal/Causal', 
    'Gantt Dependency': 'Temporal/Causal',
    'dependent': 'Temporal/Causal',
           
    'Derived': 'Workflow',
           
    'Detail': 'Workflow', 
           
    'Documentation': 'Workflow',
    'Documented': 'Workflow',
    
    'Duplicate': 'Duplication',
           
    'Epic': 'Composition', 
    'Epic-Relation': 'Composition',
    
    'Finish-to-Finish link (WBSGantt)': 'Temporal/Causal', 
    'Gantt End to End': 'Temporal/Causal', 
    'Gantt: finish-finish': 'Temporal/Causal',
    'finish-finish [GANTT]': 'Temporal/Causal', 
    
    'Gantt End to Start': 'Temporal/Causal', 
    'Gantt: finish-start': 'Temporal/Causal',
    'finish-start [GANTT]': 'Temporal/Causal',

    'Gantt Start to Start': 'Temporal/Causal', 
    
    'Gantt: start-finish': 'Temporal/Causal', 
    
    'Follows': 'Temporal/Causal', 
    'Sequence': 'Temporal/Causal', 
    
    'Implement': 'Workflow', 
    'Implements': 'Workflow', 
    
    'Issue split': 'Composition',
    'Split': 'Composition',
    'Work Breakdown': 'Composition',
    
    'Preceded By': 'Temporal/Causal', 
    
    'Reference': 'General Relation',
    'Relate': 'General Relation',
    'Related': 'General Relation', 
    'Relates': 'General Relation',
    'Relationship': 'General Relation',
    
    'Regression': 'Workflow', 
    
    'Replacement': 'Duplication',
    
    'Required': 'Temporal/Causal', 
    
    'Supercedes': 'Workflow',
    'Supersede': 'Workflow',
    'Supersession': 'Workflow', 
    
    'Test': 'Workflow', 
    'Tested': 'Workflow',
    
    'Trigger': 'Workflow',
           
    'Subtask': 'Composition'}

In [5]:
source_lt_dict={'General Relation': ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Duplication':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Composition':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Temporal/Causal':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Workflow':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring'],
              }

In [6]:
def load_data(source):
    #Loading Issues
    filename = '../../data/crawl/issues_'+source.lower()+'.csv'
    issues = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    
    #Loading Links
    filename = '../../data/crawl/clean_links_'+source.lower()+'.csv'
    links = pd.read_csv(filename, encoding="UTF-8", low_memory=False, index_col=0, sep=';')
    
    return issues, links

In [7]:
link_dict = {}
issue_dict = {}

for s in SOURCES:
    print(s.upper())
    issues, links = load_data(s)
    link_dict[s] = links
    issue_dict[s] = issues
    print(f'Loaded {len(issues)} issues and {len(links)} links')

APACHE
Loaded 1014926 issues and 255767 links
HYPERLEDGER
Loaded 28146 issues and 16304 links
INTELDAOS
Loaded 9474 issues and 2599 links
JFROG
Loaded 15535 issues and 3229 links
JIRA
Loaded 274545 issues and 99819 links
JIRAECOSYSTEM
Loaded 41866 issues and 11398 links
MARIADB
Loaded 31229 issues and 14618 links
MINDVILLE
Loaded 2134 issues and 44 links
MONGODB
Loaded 137172 issues and 63821 links
QT
Loaded 148579 issues and 40105 links
REDHAT
Loaded 353000 issues and 119669 links
SAKAI
Loaded 50550 issues and 19803 links
SECONDLIFE
Loaded 1867 issues and 631 links
SONATYPE
Loaded 87284 issues and 4465 links
SPRING
Loaded 69156 issues and 14462 links


In [8]:
def add_mean(df):
    
    og_df = df
    
    averages = og_df.mean()
    avg_row = averages.to_list()
    avg_row.insert(0, "Mean")
    
    stds = og_df.std()
    std_row = stds.to_list()
    std_row.insert(0, "Standard Dev.")
    
    df.loc[len(df)] = avg_row
    df.loc[len(df)] = std_row
    
    return df

In [9]:
def calc_graph_structures(issues, links):
    num_issues = len(set(issues['issue_id']))
    
    G = nx.Graph()

    print("Add Edges")
    for i in tqdm(links.index):
        G.add_edge(links['issue_id_1'].iloc[i], links['issue_id_2'].iloc[i])

    num_orphans = num_issues-G.number_of_nodes()

    components = nx.connected_components(G)
    
    assortativity = nx.degree_assortativity_coefficient(G)
    assortativity = round(assortativity, 3)

    num_comp = 0
    sum_den = 0
    sum_diam = 0
    cycle_count = 0
    star_count = 0
    tree_count = 0

    count_2_component = 0
    count_3m_component = 0
    print("Load Components")
    for c in tqdm(components):
        SG = nx.subgraph(G, c)
        n = SG.number_of_nodes()
        num_comp += 1
        if n==2:
            count_2_component += 1
        else:
            count_3m_component+=1
            diam = nx.diameter(SG)
            den = nx.density(SG)
            sum_diam += diam
            sum_den += den
            try:
                cycles = list(nx.find_cycle(SG, orientation="ignore"))
                cycle_count+=1
            except:
                tree_count+=1
                if (diam == 2) & (n>=3) & (SG.number_of_edges()==n-1):
                    star_count+=1
        
    num_issues_in_complex = round(((num_issues-num_orphans-2*count_2_component)/num_issues)*100,2)
    try:
        share_complex_components = round(((count_3m_component)/num_comp)*100,2)
    except:
        share_complex_components = 0
        
    per_orphans = round((num_orphans/num_issues)*100,2)
    
    avg_diam = sum_diam/count_3m_component
    avg_diam = round(avg_diam, 3)
    avg_dens = sum_den/count_3m_component
    avg_dens = round(avg_dens, 3)
    per_cycles = round((cycle_count/count_3m_component)*100,2)
    per_tree = round((tree_count/count_3m_component)*100,2)
    per_star = round((star_count/count_3m_component)*100,2)
    try: 
        per_2_comp = round((count_2_component/num_comp)*100,2)
        per_3_comp = round((count_3m_component/num_comp)*100,2)
    except:
        per_2_comp = 0
        per_3_comp = 0
    
    return num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star

In [10]:
overall_struc_df = pd.DataFrame(columns=['source', 'num_issues', 'num_orphans', 'per_orphans', 'num_comp', 'count_2_component', 'per_2_comp', 'count_3m_component', 'per_3_comp', 'num_issues_in_complex', 'share_complex_components', 'assortativity', 'avg_diam', 'avg_dens', 'per_cycles', 'per_tree', 'per_star'])
j = 0

for s in ['Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

HYPERLEDGER
Add Edges


100%|█████████████████████████████████████████████████████████████████████████| 16304/16304 [00:00<00:00, 87186.93it/s]


Load Components


1392it [01:53, 12.21it/s] 


INTELDAOS
Add Edges


...

3217it [00:03, 811.35it/s] 


MARIADB
Add Edges


100%|█████████████████████████████████████████████████████████████████████████| 14618/14618 [00:00<00:00, 89682.10it/s]


Load Components


2085it [02:17, 15.16it/s]


In [11]:
overall_struc_df

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,28146,12689,45.08,1392,659,47.34,733,52.66,50.23,52.66,-0.112,2.679,0.427,21.69,78.31,62.07
1,IntelDAOS,9474,6556,69.20,605,354,58.51,251,41.49,23.33,41.49,-0.074,2.869,0.507,21.51,78.49,58.96
2,JFrog,15535,11096,71.43,1277,845,66.17,432,33.83,17.70,33.83,-0.147,2.447,0.557,12.73,87.27,66.44
3,Jira,274535,146199,53.25,40754,32044,78.63,8710,21.37,23.40,21.37,0.000,2.692,0.535,34.74,65.26,47.70
4,JiraEcosystem,41866,28062,67.03,3217,1916,59.56,1301,40.44,23.82,40.44,-0.152,2.553,0.494,17.68,82.32,63.03
5,MariaDB,31228,17333,55.50,2085,1257,60.29,828,39.71,36.44,39.71,-0.061,2.518,0.543,24.88,75.12,57.13


In [12]:
for s in ['Mindville', 'MongoDB', 'Qt']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

MINDVILLE
Add Edges


100%|███████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 43950.79it/s]


Load Components


41it [00:00, 41081.33it/s]


MONGODB
Add Edges


...

9520it [53:52,  2.94it/s]


QT
Add Edges


100%|█████████████████████████████████████████████████████████████████████████| 40105/40105 [00:00<00:00, 71745.07it/s]


Load Components


8072it [14:51,  9.06it/s]


In [13]:
overall_struc_df

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,28146,12689,45.08,1392,659,47.34,733,52.66,50.23,52.66,-0.112,2.679,0.427,21.69,78.31,62.07
1,IntelDAOS,9474,6556,69.20,605,354,58.51,251,41.49,23.33,41.49,-0.074,2.869,0.507,21.51,78.49,58.96
2,JFrog,15535,11096,71.43,1277,845,66.17,432,33.83,17.70,33.83,-0.147,2.447,0.557,12.73,87.27,66.44
3,Jira,274535,146199,53.25,40754,32044,78.63,8710,21.37,23.40,21.37,0.000,2.692,0.535,34.74,65.26,47.70
4,JiraEcosystem,41866,28062,67.03,3217,1916,59.56,1301,40.44,23.82,40.44,-0.152,2.553,0.494,17.68,82.32,63.03
5,MariaDB,31228,17333,55.50,2085,1257,60.29,828,39.71,36.44,39.71,-0.061,2.518,0.543,24.88,75.12,57.13
6,Mindville,2134,2049,96.02,41,39,95.12,2,4.88,0.33,4.88,-0.055,2.000,0.583,0.00,100.00,100.00
7,MongoDB,137150,75200,54.83,9520,5894,61.91,3626,38.09,36.57,38.09,-0.009,2.690,0.525,19.75,80.25,56.07
8,Qt,148578,103768,69.84,8072,5136,63.63,2936,36.37,23.25,36.37,-0.160,2.448,0.521,14.61,85.39,65.97


In [14]:
for s in ['RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

REDHAT
Add Edges


100%|███████████████████████████████████████████████████████████████████████| 119669/119669 [00:01<00:00, 79237.67it/s]


Load Components


31484it [34:19, 15.28it/s]  


SAKAI
Add Edges


...

1806it [00:00, 7371.46it/s] 


SPRING
Add Edges


100%|█████████████████████████████████████████████████████████████████████████| 14462/14462 [00:00<00:00, 87649.14it/s]


Load Components


4331it [00:10, 403.91it/s]


In [15]:
s = 'Apache'
issues = issue_dict[s]
links = link_dict[s]
num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

j+=1

Add Edges


100%|███████████████████████████████████████████████████████████████████████| 255767/255767 [00:03<00:00, 76558.55it/s]


Load Components


56838it [3:20:16,  4.73it/s]  


In [16]:
add_mean(overall_struc_df)

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,2.814600e+04,12689.000000,45.080000,1392.000000,659.000000,47.340000,733.000000,52.660000,50.230000,52.660000,-0.112000,2.679000,0.427000,21.690000,78.310000,62.070000
1,IntelDAOS,9.474000e+03,6556.000000,69.200000,605.000000,354.000000,58.510000,251.000000,41.490000,23.330000,41.490000,-0.074000,2.869000,0.507000,21.510000,78.490000,58.960000
2,JFrog,1.553500e+04,11096.000000,71.430000,1277.000000,845.000000,66.170000,432.000000,33.830000,17.700000,33.830000,-0.147000,2.447000,0.557000,12.730000,87.270000,66.440000
3,Jira,2.745350e+05,146199.000000,53.250000,40754.000000,32044.000000,78.630000,8710.000000,21.370000,23.400000,21.370000,0.000000,2.692000,0.535000,34.740000,65.260000,47.700000
4,JiraEcosystem,4.186600e+04,28062.000000,67.030000,3217.000000,1916.000000,59.560000,1301.000000,40.440000,23.820000,40.440000,-0.152000,2.553000,0.494000,17.680000,82.320000,63.030000
5,MariaDB,3.122800e+04,17333.000000,55.500000,2085.000000,1257.000000,60.290000,828.000000,39.710000,36.440000,39.710000,-0.061000,2.518000,0.543000,24.880000,75.120000,57.130000
6,Mindville,2.134000e+03,2049.000000,96.020000,41.000000,39.000000,95.120000,2.000000,4.880000,0.330000,4.880000,-0.055000,2.000000,0.583000,0.000000,100.000000,100.000000
7,MongoDB,1.371500e+05,75200.000000,54.830000,9520.000000,5894.000000,61.910000,3626.000000,38.090000,36.570000,38.090000,-0.009000,2.690000,0.525000,19.750000,80.250000,56.070000
8,Qt,1.485780e+05,103768.000000,69.840000,8072.000000,5136.000000,63.630000,2936.000000,36.370000,23.250000,36.370000,-0.160000,2.448000,0.521000,14.610000,85.390000,65.970000
9,RedHat,3.529970e+05,214496.000000,60.760000,31484.000000,19191.000000,60.950000,12293.000000,39.050000,28.360000,39.050000,-0.053000,2.640000,0.520000,20.960000,79.040000,58.670000


In [17]:
def calc_subgraph_structures(issues, links, linktype):
    num_issues = len(set(issues['issue_id']))
           
    links_lt = links[links['mappedtype']==linktype]
    links_lt.reset_index(inplace=True)

    G = nx.Graph()

    for i in links_lt.index:
        G.add_edge(links_lt['issue_id_1'].iloc[i], links_lt['issue_id_2'].iloc[i])

    num_orphans = num_issues-G.number_of_nodes()
    try:
        assortativity = nx.degree_assortativity_coefficient(G)
        assortativity = round(assortativity, 3)
    except:
        assortativity = 0

    transitivity = nx.transitivity(G)
    transitivity = round(transitivity, 3)
    
    num_links = G.number_of_edges()

    components = nx.connected_components(G)

    num_comp = 0  
    sum_den = 0
    sum_diam = 0
    cycle_count = 0
    star_count = 0
    tree_count = 0

    count_2_component = 0
    count_3m_component = 0

    for c in components:
        SG = nx.subgraph(G, c)
        n = SG.number_of_nodes()
        num_comp += 1
        if n==2:
            count_2_component += 1
        else:
            count_3m_component+=1
            diam = nx.diameter(SG)
            den = nx.density(SG)
            sum_diam += diam
            sum_den += den
            try:
                cycles = list(nx.find_cycle(SG, orientation="ignore"))
                cycle_count+=1
            except:
                tree_count+=1
                if (diam == 2) & (n>=3) & (SG.number_of_edges()==n-1):
                    star_count+=1

#     if num_comp == 0:
#         num_comp = 1
    
#     if count_3m_component == 0:
#         count_3m_component = 1
    
    num_issues_in_complex = round(((num_issues-num_orphans-2*count_2_component)/num_issues)*100,2)
    try:
        share_complex_components = round(((count_3m_component)/num_comp)*100,2)
    except:
        share_complex_components = 0
    per_orphans = round((num_orphans/num_issues)*100,2)
    
    try:
        avg_diam = sum_diam/count_3m_component
        avg_diam = round(avg_diam, 3)
        avg_dens = sum_den/count_3m_component
        avg_dens = round(avg_dens, 3)
        per_cycles = round((cycle_count/count_3m_component)*100,2)
        per_tree = round((tree_count/count_3m_component)*100,2)
        per_star = round((star_count/count_3m_component)*100,2)
    except:
        avg_diam = 0
        avg_dens = 0
        per_cycles = 0
        per_tree = 0
        per_star = 0
    
    try:
        per_2_comp = round((count_2_component/num_comp)*100,2)
        per_3_comp = round((count_3m_component/num_comp)*100,2)
    except:
        per_2_comp = 0
        per_3_comp = 0

    return num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links

In [18]:
linktype_metrics = {}

for lt in categories:
    print(lt.upper())
    lt_struc_df = pd.DataFrame(columns=['source', 'num_issues', 'num_orphans', 'per_orphans', 'num_comp', 'count_2_component', 'per_2_comp', 'count_3m_component', 'per_3_comp', 'num_issues_in_complex', 'share_complext_components', 'assortativity', 'transitivity', 'avg_diam', 'avg_dens', 'per_cycles', 'per_tree', 'per_star', 'num_edges'])
    j = 0
    source_list = source_lt_dict[lt]
    
    total_links = 0
    
    for s in source_list:
        issues = issue_dict[s]
        links = link_dict[s]
        
        links['mappedtype'] = links['linktype'].map(type_dict)
        
        num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links = calc_subgraph_structures(issues, links, lt)
    
        lt_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links]
    
        j+=1
        
        total_links += num_links
    
    print(total_links)    
    link_dict[lt] = lt_struc_df
        

GENERAL RELATION
225138
DUPLICATION
102395
TEMPORAL/CAUSAL
95755
COMPOSITION
210335
WORKFLOW
16290


In [19]:
for lt in categories:
    add_mean(link_dict[lt])

In [20]:
def get_latex_print(df):
    
    temp = df[['source', 'per_orphans', 'per_2_comp', 'per_3_comp',
#                'transitivity', 
               'assortativity',
               'avg_dens', 'per_tree', 'per_star']]
    
    temp['per_tree'] = temp['per_tree'].apply(lambda x: str(round(x,1))+'%')
    temp['per_star'] = temp['per_star'].apply(lambda x: str(round(x,1))+'%')
    temp['per_2_comp'] = temp['per_2_comp'].apply(lambda x: str(round(x,1))+'%')
    temp['per_3_comp'] = temp['per_3_comp'].apply(lambda x: str(round(x,1))+'%')
    temp['per_orphans'] = temp['per_orphans'].apply(lambda x: str(round(x,1))+'%')
    temp['assortativity'] = temp['assortativity'].apply(lambda x: round(x,3))
#     blub['transitivity'] = blub['transitivity'].apply(lambda x: round(x,3))
    temp['avg_dens'] = temp['avg_dens'].apply(lambda x: round(x,3))
    
#     blub.at[len(blub)-2,'num_orphans'] = str(blub.loc[len(blub)-2]['num_orphans']).split(".")[0]
#     blub.at[len(blub)-2,'num_comp'] = str(blub.loc[len(blub)-2]['num_comp']).split(".")[0]
#     blub.at[len(blub)-1,'num_orphans'] = str(blub.loc[len(blub)-1]['num_orphans']).split(".")[0]
#     blub.at[len(blub)-1,'num_comp'] = str(blub.loc[len(blub)-1]['num_comp']).split(".")[0]
    print(temp.to_latex(index=False))  

In [21]:
link_dict['Composition']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,1.014909e+06,889504.000000,87.640000,17089.000000,6917.000000,40.480000,10172.000000,59.520000,10.990000,59.520000,-0.122000,0.001000,2.143000,0.375000,3.340000,96.660000,90.170000,109304.000000
1,Hyperledger,2.814600e+04,16084.000000,57.140000,1049.000000,224.000000,21.350000,825.000000,78.650000,41.260000,78.650000,-0.155000,0.000000,2.215000,0.318000,0.970000,99.030000,85.090000,11031.000000
2,IntelDAOS,9.474000e+03,9094.000000,95.990000,106.000000,48.000000,45.280000,58.000000,54.720000,3.000000,54.720000,-0.343000,0.000000,2.000000,0.495000,0.000000,100.000000,100.000000,274.000000
3,JFrog,1.553500e+04,14151.000000,91.090000,176.000000,108.000000,61.360000,68.000000,38.640000,7.520000,38.640000,-0.513000,0.000000,2.000000,0.490000,0.000000,100.000000,100.000000,1208.000000
4,Jira,2.745350e+05,267640.000000,97.490000,1853.000000,1143.000000,61.680000,710.000000,38.320000,1.680000,38.320000,-0.084000,0.002000,2.172000,0.486000,4.510000,95.490000,85.920000,5103.000000
5,JiraEcosystem,4.186600e+04,35752.000000,85.400000,803.000000,231.000000,28.770000,572.000000,71.230000,13.500000,71.230000,-0.330000,0.004000,2.220000,0.375000,4.020000,95.980000,87.590000,5388.000000
6,MariaDB,3.122800e+04,27851.000000,89.190000,485.000000,241.000000,49.690000,244.000000,50.310000,9.270000,50.310000,-0.207000,0.005000,2.340000,0.395000,7.380000,92.620000,81.150000,3009.000000
7,MongoDB,1.371500e+05,124391.000000,90.700000,1111.000000,256.000000,23.040000,855.000000,76.960000,8.930000,76.960000,-0.155000,0.000000,2.076000,0.331000,0.820000,99.180000,95.790000,11789.000000
8,Qt,1.485780e+05,128129.000000,86.240000,2739.000000,1453.000000,53.050000,1286.000000,46.950000,11.810000,46.950000,-0.290000,0.001000,2.329000,0.378000,3.270000,96.730000,82.040000,17930.000000
9,RedHat,3.529970e+05,307528.000000,87.120000,9260.000000,3925.000000,42.390000,5335.000000,57.610000,10.660000,57.610000,-0.206000,0.002000,2.163000,0.443000,2.140000,97.860000,91.490000,36623.000000


In [22]:
link_dict['General Relation']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,1.014909e+06,911705.000000,89.830000,34426.000000,23709.000000,68.870000,10717.000000,31.130000,5.500000,31.130000,-0.016000,0.100000,2.593000,0.557000,16.870000,83.130000,59.840000,72342.000000
1,Hyperledger,2.814600e+04,24528.000000,87.150000,1061.000000,662.000000,62.390000,399.000000,37.610000,8.150000,37.610000,-0.050000,0.164000,2.639000,0.534000,21.050000,78.950000,60.400000,2807.000000
2,IntelDAOS,9.474000e+03,8148.000000,86.000000,388.000000,230.000000,59.280000,158.000000,40.720000,9.140000,40.720000,-0.090000,0.120000,2.684000,0.519000,21.520000,78.480000,53.800000,1022.000000
3,JFrog,1.553500e+04,14092.000000,90.710000,578.000000,428.000000,74.050000,150.000000,25.950000,3.780000,25.950000,0.066000,0.109000,2.387000,0.592000,12.000000,88.000000,66.000000,884.000000
4,Jira,2.745350e+05,177590.000000,64.690000,38858.000000,33991.000000,87.470000,4867.000000,12.530000,10.550000,12.530000,0.175000,0.131000,2.720000,0.559000,37.700000,62.300000,45.760000,63647.000000
5,JiraEcosystem,4.186600e+04,37823.000000,90.340000,1591.000000,1172.000000,73.660000,419.000000,26.340000,4.060000,26.340000,0.594000,0.447000,2.277000,0.596000,14.320000,85.680000,69.450000,2613.000000
6,MariaDB,3.122800e+04,23234.000000,74.400000,1830.000000,1168.000000,63.830000,662.000000,36.170000,18.120000,36.170000,-0.040000,0.098000,2.628000,0.563000,28.100000,71.900000,51.960000,7464.000000
7,Mindville,2.134000e+03,2099.000000,98.360000,16.000000,14.000000,87.500000,2.000000,12.500000,0.330000,12.500000,-0.137000,0.000000,2.000000,0.583000,0.000000,100.000000,100.000000,19.000000
8,MongoDB,1.371500e+05,106753.000000,77.840000,8355.000000,5344.000000,63.960000,3011.000000,36.040000,14.370000,36.040000,0.104000,0.033000,2.803000,0.543000,18.500000,81.500000,54.900000,25471.000000
9,Qt,1.485780e+05,135371.000000,91.110000,4633.000000,3300.000000,71.230000,1333.000000,28.770000,4.450000,28.770000,-0.024000,0.100000,2.546000,0.560000,17.100000,82.900000,60.990000,8990.000000


In [23]:
link_dict['Duplication']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,1.014909e+06,960741.000000,94.660000,24095.000000,20535.000000,85.230000,3560.000000,14.770000,1.290000,14.770000,-0.010000,0.032000,2.187000,0.607000,4.16000,95.840000,84.800000,30246.000000
1,Hyperledger,2.814600e+04,26273.000000,93.350000,762.000000,593.000000,77.820000,169.000000,22.180000,2.440000,22.180000,-0.022000,0.013000,2.710000,0.574000,2.37000,97.630000,73.960000,1115.000000
2,IntelDAOS,9.474000e+03,8958.000000,94.550000,228.000000,184.000000,80.700000,44.000000,19.300000,1.560000,19.300000,-0.026000,0.040000,2.136000,0.617000,4.55000,95.450000,84.090000,290.000000
3,JFrog,1.553500e+04,14308.000000,92.100000,562.000000,480.000000,85.410000,82.000000,14.590000,1.720000,14.590000,-0.010000,0.112000,2.012000,0.643000,6.10000,93.900000,90.240000,670.000000
4,Jira,2.745350e+05,236835.000000,86.270000,13809.000000,9792.000000,70.910000,4017.000000,29.090000,6.600000,29.090000,-0.069000,0.059000,2.249000,0.555000,9.04000,90.960000,76.700000,24551.000000
5,JiraEcosystem,4.186600e+04,38522.000000,92.010000,1419.000000,1149.000000,80.970000,270.000000,19.030000,2.500000,19.030000,-0.046000,0.030000,2.107000,0.594000,4.44000,95.560000,87.780000,1942.000000
6,MariaDB,3.122800e+04,28864.000000,92.430000,1012.000000,801.000000,79.150000,211.000000,20.850000,2.440000,20.850000,-0.043000,0.115000,2.052000,0.616000,8.06000,91.940000,83.890000,1374.000000
7,Mindville,2.134000e+03,2086.000000,97.750000,24.000000,24.000000,100.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,24.000000
8,MongoDB,1.371500e+05,122085.000000,89.020000,6309.000000,4975.000000,78.860000,1334.000000,21.140000,3.730000,21.140000,-0.051000,0.019000,2.184000,0.578000,2.32000,97.680000,84.560000,8789.000000
9,Qt,1.485780e+05,137056.000000,92.250000,4739.000000,3610.000000,76.180000,1129.000000,23.820000,2.900000,23.820000,-0.054000,0.045000,2.133000,0.589000,5.31000,94.690000,84.150000,6859.000000


In [24]:
link_dict['Temporal/Causal']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,1.014909e+06,963878.000000,94.970000,17313.000000,12304.000000,71.070000,5009.000000,28.930000,2.600000,28.930000,-0.022000,0.041000,2.481000,0.538000,10.860000,89.140000,68.300000,35284.000000
1,Hyperledger,2.814600e+04,26294.000000,93.420000,580.000000,369.000000,63.620000,211.000000,36.380000,3.960000,36.380000,-0.115000,0.074000,2.502000,0.514000,18.480000,81.520000,61.140000,1344.000000
2,IntelDAOS,9.474000e+03,8476.000000,89.470000,276.000000,172.000000,62.320000,104.000000,37.680000,6.900000,37.680000,-0.104000,0.049000,2.827000,0.498000,19.230000,80.770000,60.580000,781.000000
3,JFrog,1.553500e+04,15074.000000,97.030000,201.000000,167.000000,83.080000,34.000000,16.920000,0.820000,16.920000,0.089000,0.032000,2.118000,0.572000,5.880000,94.120000,85.290000,265.000000
4,Jira,2.745350e+05,269473.000000,98.160000,2105.000000,1676.000000,79.620000,429.000000,20.380000,0.620000,20.380000,-0.033000,0.007000,2.172000,0.595000,5.360000,94.640000,83.920000,2985.000000
5,JiraEcosystem,4.186600e+04,39719.000000,94.870000,859.000000,644.000000,74.970000,215.000000,25.030000,2.050000,25.030000,0.333000,0.306000,2.326000,0.576000,10.230000,89.770000,68.370000,1351.000000
6,MariaDB,3.122800e+04,27835.000000,89.130000,774.000000,506.000000,65.370000,268.000000,34.630000,7.620000,34.630000,-0.116000,0.006000,2.601000,0.520000,10.450000,89.550000,70.150000,2771.000000
7,Mindville,2.134000e+03,2132.000000,99.910000,1.000000,1.000000,100.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8,MongoDB,1.371500e+05,117956.000000,86.010000,4503.000000,2813.000000,62.470000,1690.000000,37.530000,9.890000,37.530000,-0.168000,0.036000,2.840000,0.470000,17.510000,82.490000,60.360000,15714.000000
9,Qt,1.485780e+05,141083.000000,94.960000,1573.000000,997.000000,63.380000,576.000000,36.620000,3.700000,36.620000,-0.180000,0.007000,2.594000,0.472000,10.590000,89.410000,72.920000,6274.000000


In [25]:
link_dict['Workflow']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,1.014909e+06,999793.000000,98.510000,6568.000000,5470.000000,83.280000,1098.000000,16.720000,0.410000,16.720000,-0.015000,0.006000,2.133000,0.596000,2.820000,97.180000,87.340000,8591.000000
1,Hyperledger,2.814600e+04,28133.000000,99.950000,6.000000,5.000000,83.330000,1.000000,16.670000,0.010000,16.670000,-0.167000,0.000000,2.000000,0.667000,0.000000,100.000000,100.000000,7.000000
2,IntelDAOS,9.474000e+03,9443.000000,99.670000,14.000000,12.000000,85.710000,2.000000,14.290000,0.070000,14.290000,-0.155000,0.000000,2.000000,0.583000,0.000000,100.000000,100.000000,17.000000
3,JFrog,1.553500e+04,15156.000000,97.560000,177.000000,158.000000,89.270000,19.000000,10.730000,0.410000,10.730000,-0.080000,0.000000,2.053000,0.618000,0.000000,100.000000,94.740000,202.000000
4,Jira,2.745350e+05,269046.000000,98.000000,1971.000000,1394.000000,70.730000,577.000000,29.270000,0.980000,29.270000,-0.112000,0.004000,2.054000,0.535000,1.390000,98.610000,94.110000,3533.000000
5,JiraEcosystem,4.186600e+04,41696.000000,99.590000,68.000000,50.000000,73.530000,18.000000,26.470000,0.170000,26.470000,-0.184000,0.000000,2.167000,0.560000,0.000000,100.000000,83.330000,102.000000
6,MongoDB,1.371500e+05,133283.000000,97.180000,1811.000000,1663.000000,91.830000,148.000000,8.170000,0.390000,8.170000,-0.022000,0.000000,2.155000,0.590000,0.680000,99.320000,86.490000,2057.000000
7,Qt,1.485780e+05,148490.000000,99.940000,38.000000,31.000000,81.580000,7.000000,18.420000,0.020000,18.420000,-0.167000,0.000000,2.000000,0.571000,0.000000,100.000000,100.000000,50.000000
8,RedHat,3.529970e+05,351841.000000,99.670000,487.000000,383.000000,78.640000,104.000000,21.360000,0.110000,21.360000,-0.126000,0.000000,2.115000,0.583000,0.960000,99.040000,89.420000,670.000000
9,Sakai,5.055000e+04,50550.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
get_latex_print(link_dict['General Relation'])

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       89.8\% &      68.9\% &      31.1\% &         -0.016 &     0.557 &    83.1\% &    59.8\% \\
  Hyperledger &       87.2\% &      62.4\% &      37.6\% &         -0.050 &     0.534 &    79.0\% &    60.4\% \\
    IntelDAOS &       86.0\% &      59.3\% &      40.7\% &         -0.090 &     0.519 &    78.5\% &    53.8\% \\
        JFrog &       90.7\% &      74.0\% &      25.9\% &          0.066 &     0.592 &    88.0\% &    66.0\% \\
         Jira &       64.7\% &      87.5\% &      12.5\% &          0.175 &     0.559 &    62.3\% &    45.8\% \\
JiraEcosystem &       90.3\% &      73.7\% &      26.3\% &          0.594 &     0.596 &    85.7\% &    69.5\% \\
      MariaDB &       74.4\% &      63.8\% &      36.2\% &         -0.040 &     0.563 &    71.9\% &    52.0\% \\
    Mindville &       98.4\% &      87.5\% &     

In [27]:
get_latex_print(link_dict['Duplication'])

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       94.7\% &      85.2\% &      14.8\% &         -0.010 &     0.607 &    95.8\% &    84.8\% \\
  Hyperledger &       93.3\% &      77.8\% &      22.2\% &         -0.022 &     0.574 &    97.6\% &    74.0\% \\
    IntelDAOS &       94.5\% &      80.7\% &      19.3\% &         -0.026 &     0.617 &    95.5\% &    84.1\% \\
        JFrog &       92.1\% &      85.4\% &      14.6\% &         -0.010 &     0.643 &    93.9\% &    90.2\% \\
         Jira &       86.3\% &      70.9\% &      29.1\% &         -0.069 &     0.555 &    91.0\% &    76.7\% \\
JiraEcosystem &       92.0\% &      81.0\% &      19.0\% &         -0.046 &     0.594 &    95.6\% &    87.8\% \\
      MariaDB &       92.4\% &      79.2\% &      20.9\% &         -0.043 &     0.616 &    91.9\% &    83.9\% \\
    Mindville &       97.8\% &     100.0\% &     

In [28]:
get_latex_print(link_dict['Temporal/Causal'])

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       95.0\% &      71.1\% &      28.9\% &         -0.022 &     0.538 &    89.1\% &    68.3\% \\
  Hyperledger &       93.4\% &      63.6\% &      36.4\% &         -0.115 &     0.514 &    81.5\% &    61.1\% \\
    IntelDAOS &       89.5\% &      62.3\% &      37.7\% &         -0.104 &     0.498 &    80.8\% &    60.6\% \\
        JFrog &       97.0\% &      83.1\% &      16.9\% &          0.089 &     0.572 &    94.1\% &    85.3\% \\
         Jira &       98.2\% &      79.6\% &      20.4\% &         -0.033 &     0.595 &    94.6\% &    83.9\% \\
JiraEcosystem &       94.9\% &      75.0\% &      25.0\% &          0.333 &     0.576 &    89.8\% &    68.4\% \\
      MariaDB &       89.1\% &      65.4\% &      34.6\% &         -0.116 &     0.520 &    89.5\% &    70.2\% \\
    Mindville &       99.9\% &     100.0\% &     

In [29]:
get_latex_print(link_dict['Composition'])

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       87.6\% &      40.5\% &      59.5\% &         -0.122 &     0.375 &    96.7\% &    90.2\% \\
  Hyperledger &       57.1\% &      21.4\% &      78.7\% &         -0.155 &     0.318 &    99.0\% &    85.1\% \\
    IntelDAOS &       96.0\% &      45.3\% &      54.7\% &         -0.343 &     0.495 &   100.0\% &   100.0\% \\
        JFrog &       91.1\% &      61.4\% &      38.6\% &         -0.513 &     0.490 &   100.0\% &   100.0\% \\
         Jira &       97.5\% &      61.7\% &      38.3\% &         -0.084 &     0.486 &    95.5\% &    85.9\% \\
JiraEcosystem &       85.4\% &      28.8\% &      71.2\% &         -0.330 &     0.375 &    96.0\% &    87.6\% \\
      MariaDB &       89.2\% &      49.7\% &      50.3\% &         -0.207 &     0.395 &    92.6\% &    81.2\% \\
      MongoDB &       90.7\% &      23.0\% &     

In [30]:
get_latex_print(link_dict['Workflow'])

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       98.5\% &      83.3\% &      16.7\% &         -0.015 &     0.596 &    97.2\% &    87.3\% \\
  Hyperledger &      100.0\% &      83.3\% &      16.7\% &         -0.167 &     0.667 &   100.0\% &   100.0\% \\
    IntelDAOS &       99.7\% &      85.7\% &      14.3\% &         -0.155 &     0.583 &   100.0\% &   100.0\% \\
        JFrog &       97.6\% &      89.3\% &      10.7\% &         -0.080 &     0.618 &   100.0\% &    94.7\% \\
         Jira &       98.0\% &      70.7\% &      29.3\% &         -0.112 &     0.535 &    98.6\% &    94.1\% \\
JiraEcosystem &       99.6\% &      73.5\% &      26.5\% &         -0.184 &     0.560 &   100.0\% &    83.3\% \\
      MongoDB &       97.2\% &      91.8\% &       8.2\% &         -0.022 &     0.590 &    99.3\% &    86.5\% \\
           Qt &       99.9\% &      81.6\% &     

In [31]:
get_latex_print(overall_struc_df.sort_values(by='source'))

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       71.5\% &      62.8\% &      37.2\% &         -0.062 &     0.499 &    82.6\% &    61.8\% \\
  Hyperledger &       45.1\% &      47.3\% &      52.7\% &         -0.112 &     0.427 &    78.3\% &    62.1\% \\
    IntelDAOS &       69.2\% &      58.5\% &      41.5\% &         -0.074 &     0.507 &    78.5\% &    59.0\% \\
        JFrog &       71.4\% &      66.2\% &      33.8\% &         -0.147 &     0.557 &    87.3\% &    66.4\% \\
         Jira &       53.2\% &      78.6\% &      21.4\% &          0.000 &     0.535 &    65.3\% &    47.7\% \\
JiraEcosystem &       67.0\% &      59.6\% &      40.4\% &         -0.152 &     0.494 &    82.3\% &    63.0\% \\
      MariaDB &       55.5\% &      60.3\% &      39.7\% &         -0.061 &     0.543 &    75.1\% &    57.1\% \\
         Mean &       66.6\% &      64.0\% &     